In [5]:
import pandas as pd

api = pd.read_csv("api_tagged.csv").drop(columns=["Unnamed: 0"])
api

,Tweet,Date,Sentiment
0,current stats of delegatedonthate block find p...,2022-05-19,-0.030147
1,bbcworld for all those who be new to this work...,2022-05-19,0.252789
2,smilingpunks floor price no gas fee polygon b...,2022-05-19,0.251752
3,i be claim my free lightning sat from bitcoine...,2022-05-19,0.380444
4,washingtonpost for all those who be new to thi...,2022-05-19,0.252789
...,...,...,...
99995,bitcoin will again reach k mark at the end of ...,2022-05-18,0.160646
99996,china be now the world ndbiggest bitcoin miner...,2022-05-18,-0.090590
99997,i wonder how elsalvadors bitcoin conference co...,2022-05-18,0.232044
99998,busd inflow to exchange have increase by over ...,2022-05-18,0.011315


In [7]:
import datetime as dt

api[api["Date"] == "2022-05-19"]

,Tweet,Date,Sentiment
0,current stats of delegatedonthate block find p...,2022-05-19,-0.030147
1,bbcworld for all those who be new to this work...,2022-05-19,0.252789
2,smilingpunks floor price no gas fee polygon b...,2022-05-19,0.251752
3,i be claim my free lightning sat from bitcoine...,2022-05-19,0.380444
4,washingtonpost for all those who be new to thi...,2022-05-19,0.252789
...,...,...,...
83527,sec chair gensler threaten action against unre...,2022-05-19,-0.164377
83528,intocryptoverse bitcoin dominance at right now...,2022-05-19,0.152668
83529,bitcoin price decrease by in the last h btc ...,2022-05-19,-0.042253
83530,top cryptocurrency inout update in last hour c...,2022-05-19,-0.046096


In [9]:
btc = pd.read_csv("data/BTC-USD.csv")
btc

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-05-12,28936.734375,30032.439453,26350.490234,29047.751953,29047.751953,66989173272
1,2022-05-13,29030.910156,30924.802734,28782.330078,29283.103516,29283.103516,42841124537
2,2022-05-14,29285.642578,30192.802734,28702.910156,30101.265625,30101.265625,28579868620
3,2022-05-15,30098.585938,31308.191406,29527.740234,31305.113281,31305.113281,25835372065
4,2022-05-16,31304.375000,31305.341797,29251.884766,29862.917969,29862.917969,32613897286
5,2022-05-17,29862.408203,30694.490234,29570.302734,30425.857422,30425.857422,29101473475
6,2022-05-18,30424.478516,30618.716797,28720.271484,28720.271484,28720.271484,31285268319
7,2022-05-19,28781.103516,29690.830078,28708.955078,29687.033203,29687.033203,34817126400


In [11]:
btc.iloc[6:8]

,Date,Open,High,Low,Close,Adj Close,Volume
6,2022-05-18,30424.478516,30618.716797,28720.271484,28720.271484,28720.271484,31285268319
7,2022-05-19,28781.103516,29690.830078,28708.955078,29687.033203,29687.033203,34817126400


In [21]:
import numpy as np

avgs = []
for i in range(8):
    avgs.append((btc.iloc[i]["High"] + btc.iloc[i]["Low"]) / 2)

avgs

[28191.464843499998,
 29853.566405999998,
 29447.856445,
 30417.96582,
 30278.613281500002,
 30132.396484,
 29669.4941405,
 29199.892578]

In [14]:
btc.iloc[6]["High"]

30618.716797

In [18]:
(btc.iloc[6]["High"] + btc.iloc[6]["Low"]) / 2

29669.4941405